In [1]:
from typing import Type
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.utilities import DuckDuckGoSearchAPIWrapper

llm = ChatOpenAI(
    temperature=0.1,
)

class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query : str = Field(
        description="The Query you will search for. Example quesry: Stock Market Symbol for Apple Company"
    )

class StockMarketSymbolSearchTool(BaseTool):
    name : str ="StockMarketSymbolSearchTool"
    description : str = """
    Use this toll to find the stock market symbol for a company.
    It takes a query as an argument.
    """
    args_schema : Type[StockMarketSymbolSearchToolArgsSchema] = StockMarketSymbolSearchToolArgsSchema

    def _run(self, query):
        ddg = DuckDuckGoSearchAPIWrapper()
        return ddg.run(query)


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        StockMarketSymbolSearchTool(),
    ]
)

prompt = "Give me information on Rigetti Computing Inc's stock and help me analyze if it's a potential good investment. Also tell me what symbol does the stock have."
agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `StockMarketSymbolSearchTool` with `{'query': 'Stock Market Symbol for Rigetti Computing Inc'}`


Discover real-time Rigetti Computing, Inc. Common Stock (RGTI) stock prices, quotes, historical data, news, and Insights for informed trading and investment decisions. Stay ahead with Nasdaq. Over the last year, Rigetti Computing, Inc.'s stock price has increased by 522.49%. Rigetti Computing, Inc. is currently approximately $11.03 per share. What is Rigetti Computing, Inc.'s 2025 ... 5 days ago · Find all our analysis, stock picks and recommendations on the stock market. And much more: Today on Wall Street, the Weekly Market Update... About RGTI. Rigetti Computing, Inc., through its subsidiaries, builds quantum computers and the superconducting quantum processors. The company offers cloud in a form of quantum processing unit, such as 9-qubit chip and Ankaa-2 system under the Novera brand name; and sells access to its quantum computers thr

{'input': "Give me information on Rigetti Computing Inc's stock and help me analyze if it's a potential good investment. Also tell me what symbol does the stock have.",
 'output': "The stock market symbol for Rigetti Computing Inc is RGTI. \n\nAs of now, Rigetti Computing Inc's stock price is approximately $11.03 per share, and over the last year, the stock price has increased by 522.49%. \n\nTo analyze if Rigetti Computing Inc is a potential good investment, we can consider factors such as the company's financial performance, growth prospects, market trends, and competition in the quantum computing industry. It's important to conduct a thorough analysis and consider consulting with a financial advisor before making any investment decisions."}